In [1]:
import pandas as pd
import re
import numpy as np

In [45]:
df = pd.read_csv('./openaq.csv', encoding='utf-8',  sep=';')

In [3]:
df.head()

,Country Code,City,Location,Coordinates,Pollutant,Source Name,Unit,Value,Last Updated,Country Label
0,DE,Manfred Lotz,"Cottbus, W.-Külz-Str.","51.75413,14.325549",PM2.5,EEA Germany,µg/m³,21.7,2016-12-21T12:00:00+01:00,Germany
1,ES,Castellón/Castelló,ES1969A,"39.8236111095,-0.24444444",NO2,EEA Spain,µg/m³,1.0,2021-07-13T08:00:00+02:00,Spain
2,ES,Valencia/València,ES1885A,"39.4802777695,-0.33638888",SO2,EEA Spain,µg/m³,5.0,2021-08-06T11:00:00+02:00,Spain
3,ES,Alicante/Alacant,ES2008A,"37.9911111095,-0.69",PM2.5,EEA Spain,µg/m³,9.0,2021-08-06T11:00:00+02:00,Spain
4,ES,Alicante/Alacant,ES1968A,"38.3511111095,-0.51388888",PM10,EEA Spain,µg/m³,1.0,2021-08-06T11:00:00+02:00,Spain


### Supprimer les données qui ne concernent pas la journée du 06-08-2021

In [4]:
df['Last Updated'] = df['Last Updated'].str[:10]

df = df.loc[df['Last Updated'] == "2021-08-06"]
df

,Country Code,City,Location,Coordinates,Pollutant,Source Name,Unit,Value,Last Updated,Country Label
2,ES,Valencia/València,ES1885A,"39.4802777695,-0.33638888",SO2,EEA Spain,µg/m³,5.000,2021-08-06,Spain
3,ES,Alicante/Alacant,ES2008A,"37.9911111095,-0.69",PM2.5,EEA Spain,µg/m³,9.000,2021-08-06,Spain
4,ES,Alicante/Alacant,ES1968A,"38.3511111095,-0.51388888",PM10,EEA Spain,µg/m³,1.000,2021-08-06,Spain
5,ES,Valencia/València,ES1911A,"39.7052777695,-0.33666666",CO,EEA Spain,µg/m³,100.000,2021-08-06,Spain
6,ES,Valencia/València,ES1185A,"39.6672222195,-0.23472222",PM10,EEA Spain,µg/m³,13.000,2021-08-06,Spain
...,...,...,...,...,...,...,...,...,...,...
40408,TH,Bangkok,"Din Dang, Bangkok","13.7619223,100.558606",PM10,Air4Thai,µg/m³,42.000,2021-08-06,Thailand
40409,TH,Samut Prakan,"City Hall, Samut Prakan","13.599149,100.597345",SO2,Air4Thai,ppm,0.000,2021-08-06,Thailand
40410,TH,Samut Prakan,"City Hall, Samut Prakan","13.599149,100.597345",NO2,Air4Thai,ppm,0.006,2021-08-06,Thailand
40411,TH,Chachoengsao,Municipality Office Tungsadao,"13.588554,101.286359",PM10,Air4Thai,µg/m³,37.000,2021-08-06,Thailand


### Supprimer les données qui sont illisibles et pour lesquelles on ne comprend pas la ville concernée.

In [5]:
df.City.isnull().value_counts()

False    24204
True        83
Name: City, dtype: int64

In [6]:
df["City"] = df.City.dropna()
df.City.dropna().isnull().value_counts()


False    24204
Name: City, dtype: int64

In [7]:
col_cities = df.City.tolist()
print(col_cities)

['Valencia/València', 'Alicante/Alacant', 'Alicante/Alacant', 'Valencia/València', 'Valencia/València', 'Alicante/Alacant', 'Almería', 'Málaga', 'Sevilla', 'Cádiz', 'Huelva', 'Huelva', 'Huelva', 'Huelva', 'Madrid', 'Araba/Álava', 'Bizkaia', 'Granada', 'Madrid', 'Badajoz', 'Asturias', 'Madrid', 'Madrid', 'Bizkaia', 'Asturias', 'Asturias', 'Asturias', 'Illes Balears', 'Cantabria', 'Cantabria', 'Cantabria', 'Cantabria', 'Cantabria', 'Cantabria', 'Cantabria', 'Guadalajara', 'La Rioja', 'Meurthe-et-Moselle', 'Somme', 'Deux-Sèvres', 'Nord', 'Rhône', "Côte-d'Or", 'Ain', 'Ain', 'Gladstone', 'Townsville', 'Melbourne', 'Melbourne', 'Traralgon', 'Geelong', 'Central Queensland', 'Brussels-Capital Region', 'Antwerpen', 'Antwerpen', 'Antwerpen', 'Vlaams-Brabant', 'Brabant wallon', 'Hainaut', 'Antwerpen', 'Oost-Vlaanderen', 'София (столица)', 'Diadema', 'Diadema', 'Jacareí', 'São Paulo', 'Americana', 'Limeira', 'Marília', 'Ribeirão Preto', 'BRITISH COLUMBIA', 'ONTARIO', 'NOVA SCOTIA', 'NEWFOUNDLAND',

### Delete Chinese Cities

In [8]:
df['City'] = df['City'].str.replace(r'[^\x00-\x7F]+', '')
df.loc[df['City'] == ""]

<ipython-input-8-9f63f61ec5ae>:1: FutureWarning: The default value of regex will change from True to False in a future version.
  df['City'] = df['City'].str.replace(r'[^\x00-\x7F]+', '')


,Country Code,City,Location,Coordinates,Pollutant,Source Name,Unit,Value,Last Updated,Country Label
496,CN,,上街区政府,"34.803229,113.30952",CO,ChinaAQIData,µg/m³,800.0,2021-08-06,China
497,CN,,中心小学,"47.4346,128.4741",PM10,ChinaAQIData,µg/m³,14.0,2021-08-06,China
498,CN,,中海酒店,"37.3956,117.9378",O3,ChinaAQIData,µg/m³,189.0,2021-08-06,China
499,CN,,丰台小屯,"39.8768,116.2545",CO,ChinaAQIData,µg/m³,700.0,2021-08-06,China
500,CN,,九州社区,"36.4361,116.0445",CO,ChinaAQIData,µg/m³,900.0,2021-08-06,China
...,...,...,...,...,...,...,...,...,...,...
40013,CN,,嘉兴学院,"30.7478,120.726",SO2,ChinaAQIData,µg/m³,3.0,2021-08-06,China
40014,CN,,园林处,"43.8805,125.3297",SO2,ChinaAQIData,µg/m³,3.0,2021-08-06,China
40015,CN,,圭峰西,"22.5311,113.0219",CO,ChinaAQIData,µg/m³,700.0,2021-08-06,China
40016,CN,,场坝街,"25.0925,104.9022",O3,ChinaAQIData,µg/m³,145.0,2021-08-06,China


In [9]:
df['City'] = df['City'].replace(['', 'unused'], np.NaN)
df.loc[df['City'] == ""]

,Country Code,City,Location,Coordinates,Pollutant,Source Name,Unit,Value,Last Updated,Country Label


In [10]:
df = df.dropna()
df

,Country Code,City,Location,Coordinates,Pollutant,Source Name,Unit,Value,Last Updated,Country Label
2,ES,Valencia/Valncia,ES1885A,"39.4802777695,-0.33638888",SO2,EEA Spain,µg/m³,5.000,2021-08-06,Spain
3,ES,Alicante/Alacant,ES2008A,"37.9911111095,-0.69",PM2.5,EEA Spain,µg/m³,9.000,2021-08-06,Spain
4,ES,Alicante/Alacant,ES1968A,"38.3511111095,-0.51388888",PM10,EEA Spain,µg/m³,1.000,2021-08-06,Spain
5,ES,Valencia/Valncia,ES1911A,"39.7052777695,-0.33666666",CO,EEA Spain,µg/m³,100.000,2021-08-06,Spain
6,ES,Valencia/Valncia,ES1185A,"39.6672222195,-0.23472222",PM10,EEA Spain,µg/m³,13.000,2021-08-06,Spain
...,...,...,...,...,...,...,...,...,...,...
40408,TH,Bangkok,"Din Dang, Bangkok","13.7619223,100.558606",PM10,Air4Thai,µg/m³,42.000,2021-08-06,Thailand
40409,TH,Samut Prakan,"City Hall, Samut Prakan","13.599149,100.597345",SO2,Air4Thai,ppm,0.000,2021-08-06,Thailand
40410,TH,Samut Prakan,"City Hall, Samut Prakan","13.599149,100.597345",NO2,Air4Thai,ppm,0.006,2021-08-06,Thailand
40411,TH,Chachoengsao,Municipality Office Tungsadao,"13.588554,101.286359",PM10,Air4Thai,µg/m³,37.000,2021-08-06,Thailand


In [11]:
col_cities = df.City.tolist()
#print(col_cities)
df = df.dropna()
df.City.value_counts()


Bayern                      160
Nordrhein-Westfalen         157
Baden-Wrttemberg            141
Hessen                      129
Niedersterreich             116
                           ... 
CHURCHILL                     1
Lake Havasu City-Kingman      1
Sandve                        1
Kronoberg                     1
East Stroudsburg              1
Name: City, Length: 1828, dtype: int64

### Pour chaque polluant, vérifier qu'on n'a qu'une seule mesure utilisée. Autrement, supprimer les données qui correspondent à la mesure minoritaire ou moins cohérente.


In [51]:
df1 = df.groupby(['City'])["Value"]
df1